# Restrict teams to their access groups

This robot notebook deletes a set of members from the club's TeamApp. This is used to delete all members that are not part of the new season.

Demo of bulk editing for recurring schedules: https://youtu.be/Co0hPJ_xdbc

[![https://www.youtube.com/watch?v=Co0hPJ_xdbc](https://img.youtube.com/vi/Co0hPJ_xdbc/0.jpg)](https://www.youtube.com/watch?v=Co0hPJ_xdbc)

In [ ]:
import time

# Download geckodriver (https://github.com/mozilla/geckodriver/releases) and put it in path
# Salenium webdriver: https://www.selenium.dev/documentation/overview/
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select


In [ ]:
def report_element(e, i=""):
    print(i, e)
    print(i, e.text)
    print(i, e.get_attribute("class"))
    print(i, e.id)
    print(i, e.tag_name)

def report_elements(browser, search_string):    # get all the three dots edit options
    print(search_string)
    for i, button in enumerate(browser.find_elements(By.XPATH, search_string)):
        report_element(button, i)
        print()

## Create Virtual Browser

Create a firefox instance using an existing profile that MUST be logged into TeamApp.

Set the following variables and in particular `FIREFOX_PROFILE_PATH` to the existing Firefox profile that is already logged into the club's TeamApp.

In [ ]:
# URL of the club's TeamApp
URL_TEAMAPP = "https://brunswickmagicbasketball.teamapp.com"

# Folder with the firefox profile to be used
FIREFOX_PROFILE_PATH = '/home/ssardina/.mozilla/firefox/88weweww.ssardina'

GECKODRIVER_PATH = '/home/ssardina/bin/geckodriver'

URL_MODULE = f"{URL_TEAMAPP}/teams?_list=v1"

In [ ]:
# firefox profile of user already logged in teamapp

options = Options()
# options.headless = True
options.page_load_strategy = 'eager'
options.set_preference('profile', FIREFOX_PROFILE_PATH)

# https://github.com/SeleniumHQ/selenium/issues/11028
# use an existing firefox profile (must be logged in teamapps already!)
options.add_argument("-profile")
options.add_argument(FIREFOX_PROFILE_PATH)

service = None
# service = Service(GECKODRIVER_PATH)

if service is not None:
    browser = webdriver.Firefox(service=service, options=options)
else:
    browser = webdriver.Firefox(options=options)
# browser.maximize_window()

browser.get(URL_MODULE)

# browser.quit()


In case you want to close it:

In [ ]:
browser.quit()

# Restrict one team access group

We start by retrieving the team list:

In [ ]:
# get to the All Schedule Entries
browser.get(URL_MODULE)

In [ ]:
report_elements(browser, r"//*[@class='v-list-item__title']")

### 1. Go to the team schedule and edit

First enter into the team schedule:

In [ ]:
# the team number you want to edit
TEAM_NO = 0

# get the button of all the training schedules in the screen (listed with names "U08 Mixed Gold Training")
buttons = browser.find_elements(By.XPATH, r"//*[@class='v-list-item__title']")

access_group = buttons[TEAM_NO].text
buttons[TEAM_NO].click()
print("Team to edit:" , access_group)

Next click the three dot options top right and then EDIT:

In [ ]:
# click options three dots button (top-right)
buttons = browser.find_elements(By.XPATH, '//*[contains(text(), "more_vert")]')
buttons[0].click()

# now EDIT!
buttons = browser.find_elements(By.XPATH, "//*[contains(text(),'Edit') and @class='v-list-item__title']")
# report_elements(browser, "//*[contains(text(),'Edit')]")
buttons[0].click()

In [ ]:
# scroll-down to the bottom of page
browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")

### 2. Now edit the open schedule to recurr

Now we edit the schedule to recurr.

In [ ]:
# select Access Group option
buttons = browser.find_elements(By.XPATH, "//*[contains(text(),'Access Groups')]")
buttons[0].click()

report_elements(browser, "//*[contains(text(),'Access Groups')]")


Select the correct access group for the team:

In [ ]:
# click arrow to show all dropbox
buttons = browser.find_elements(By.XPATH, "//i[contains(@class, 'mdi-menu-down')]")
buttons[0].click()

# delete all that is there already and enter the access group team name string to filter
buttons = browser.find_elements(By.XPATH, "//input[@name = 'team[access_level_ids_csv]']")
buttons[0].send_keys(Keys.BACKSPACE * 2)
buttons[0].send_keys(access_group)

# click the access group
buttons = browser.find_elements(By.XPATH, f"//span[contains(text(), '{access_group}')]")
buttons[0].click()

### 3. Save changes

In [ ]:
# click SAVE button
buttons = browser.find_elements(By.XPATH, "//*[contains(text(), 'Save') and contains(@class, 'v-btn__content')]")
buttons[0].click()

In [ ]:
# click Ok button
buttons = browser.find_elements(By.XPATH, "//*[contains(text(), 'Ok') and contains(@class, 'v-btn__content')]")
buttons[0].click()

Calendar should come now. We need to navigate to the month and then select the day.

# Bulk edit teams to restrict access groups

First initialize the schedule full page you want to process. There may be many pages to go through.. :-)

In [ ]:
URL_MODULE = f"{URL_TEAMAPP}/teams?_list=v1"

In [ ]:
# get to the All Schedule Entries
browser.get(URL_MODULE)
NO_TEAMS = len(browser.find_elements(By.XPATH, r"//*[@class='v-list-item__title']"))

print("Number of teams to update (in page):", NO_TEAMS)

for i in range(10,NO_TEAMS):
    browser.get(URL_MODULE)

    time.sleep(3)

    # get the button of all the training schedules in the screen (listed with names "U08 Mixed Gold Training")
    buttons = browser.find_elements(By.XPATH, r"//*[@class='v-list-item__title']")

    access_group = buttons[i].text
    print(f"Team to edit {i}:" , access_group)
    buttons[i].click()

    time.sleep(2)

    # click options three dots button (top-right)
    buttons = browser.find_elements(By.XPATH, '//*[contains(text(), "more_vert")]')
    buttons[0].click()

    # now EDIT!
    buttons = browser.find_elements(By.XPATH, "//*[contains(text(),'Edit') and @class='v-list-item__title']")
    # report_elements(browser, "//*[contains(text(),'Edit')]")
    buttons[0].click()

    # scroll-down to the bottom of page
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight + 100)")

    time.sleep(1)
    # select Access Group option
    buttons = browser.find_elements(By.XPATH, "//*[contains(text(),'Access Groups')]")
    buttons[0].click()


    time.sleep(1)
    # click arrow to show all dropbox
    buttons = browser.find_elements(By.XPATH, "//i[contains(@class, 'mdi-menu-down')]")
    buttons[0].click()

    # delete all that is there already and enter the access group team name string to filter
    buttons = browser.find_elements(By.XPATH, "//input[@name = 'team[access_level_ids_csv]']")
    buttons[0].send_keys(Keys.BACKSPACE * 2)
    buttons[0].send_keys(access_group)

    # click the access group
    buttons = browser.find_elements(By.XPATH, f"//span[contains(text(), '{access_group}')]")
    buttons[0].click()

    # click SAVE button
    buttons = browser.find_elements(By.XPATH, "//*[contains(text(), 'Save') and contains(@class, 'v-btn__content')]")
    buttons[0].click()

    time.sleep(1)
    # click Ok button
    buttons = browser.find_elements(By.XPATH, "//*[contains(text(), 'Ok') and contains(@class, 'v-btn__content')]")
    buttons[0].click()

    print("\t Team saved!", access_group)

    # wait 3 seconds for next schedule
    time.sleep(2)
